Collecting IMBD dataset


In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.1MB/s]


In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Reversing text

In [4]:
print(vars(train_data.examples[1]))

{'text': ['Timeless', 'musical', 'gem', ',', 'with', 'Gene', 'Kelly', 'in', 'top', 'form', ',', 'stylish', 'direction', 'by', 'Vincente', 'Minnelli', ',', 'and', 'wonderful', 'musical', 'numbers', '.', 'It', 'is', 'great', 'entertainment', 'from', 'start', 'to', 'finish', ',', 'one', 'of', 'those', 'films', 'that', 'people', 'watch', 'with', 'a', 'smile', 'and', 'say', '"', 'they', 'do', "n't", 'make', "'em", 'like', 'they', 'used', 'to', '!', '"', 'But', 'they', 'never', 'did', 'quite', 'make', 'them', 'like', 'this', '.', 'The', 'climactic', '25', 'minute', 'musical', 'sequence', 'without', 'any', 'dialogue', 'is', 'among', 'the', 'most', 'beautiful', 'in', 'film', 'history', '.', 'Movie', 'magic', ',', 'clearly', 'derived', 'from', 'the', 'heart', 'and', 'soul', 'of', 'everyone', 'involved', '.', 'A', 'must', 'see', '!'], 'label': 'pos'}


In [5]:
for index in range(len(train_data)):
  vars(train_data.examples[index]). get('text'). reverse()


{'text': ['.', 'waste', 'a', 'What', '!', 'time', 'the', 'me', 'give', 'but', ',', 'money', 'the', 'keep', 'can', 'They', '.', 'back', 'life', 'my', 'of', 'minutes', 'ninety', 'that', 'want', 'I', ',', 'line', '/>Bottom', '/><br', 'br', '<', '.', 'either', 'good', "n't", 'were', 'really', 'they', 'but', ',', 'bad', "n't", 'were', 'creatures', 'The', '.', 'film', 'entire', 'the', 'out', 'through', 'off', 'was', 'continuity', '/>The', '/><br', 'br', '<', '.', 'comebacks', 'their', 'stage', 'and', 'try', 'to', 'film', 'lousy', 'a', 'picked', 'Dorff', 'and', 'Slater', '.', 'archaeologist', 'an', 'as', 'unbelievable', 'VERY', 'was', 'she', 'but', 'Reid', 'like', 'I', ',', 'sorry', "'m", 'I', 'And', '.', 'robot', 'a', 'from', 'if', 'as', 'delivered', 'were', 'Lines', '.', 'Dorff', 'and', 'Reid', ',', 'Slater', 'of', 'likes', 'the', 'from', 'more', 'expect', 'You', '.', 'inexperience', 'of', 'sense', 'a', 'off', 'gave', 'actors', 'the', 'and', 'stiff', 'was', 'acting', '/>The', '/><br', 'poss

In [6]:
print(vars(train_data.examples[1]))

{'text': ['!', 'see', 'must', 'A', '.', 'involved', 'everyone', 'of', 'soul', 'and', 'heart', 'the', 'from', 'derived', 'clearly', ',', 'magic', 'Movie', '.', 'history', 'film', 'in', 'beautiful', 'most', 'the', 'among', 'is', 'dialogue', 'any', 'without', 'sequence', 'musical', 'minute', '25', 'climactic', 'The', '.', 'this', 'like', 'them', 'make', 'quite', 'did', 'never', 'they', 'But', '"', '!', 'to', 'used', 'they', 'like', "'em", 'make', "n't", 'do', 'they', '"', 'say', 'and', 'smile', 'a', 'with', 'watch', 'people', 'that', 'films', 'those', 'of', 'one', ',', 'finish', 'to', 'start', 'from', 'entertainment', 'great', 'is', 'It', '.', 'numbers', 'musical', 'wonderful', 'and', ',', 'Minnelli', 'Vincente', 'by', 'direction', 'stylish', ',', 'form', 'top', 'in', 'Kelly', 'Gene', 'with', ',', 'gem', 'musical', 'Timeless'], 'label': 'pos'}


In [7]:

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:54, 2.08MB/s]                           
 99%|█████████▉| 397955/400000 [00:16<00:00, 23351.95it/s]

In [8]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnns = nn.ModuleList()

        for index in range(n_layers):
          if index == 0:
            input_size = embedding_dim
          else:
            input_size = hidden_dim

          self.rnns.append(nn.LSTM(input_size, hidden_dim, 1, bidirectional=bidirectional, 
                           dropout=dropout))          
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        #packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        for index in range(self.n_layers):
          if(index == 0):
            t1 = packed_embedded
          else:
            t1 = hidden
          packed_output, (hidden, cell) = self.rnns[index](t1)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [11]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,339,241 trainable parameters


In [12]:

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [13]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2281,  0.8470, -0.4455,  ..., -0.1672, -0.4080,  0.0305],
        [-0.5568,  0.5786,  0.5510,  ..., -0.0846, -0.2132,  0.3013],
        [ 0.1659, -0.4335, -0.3923,  ..., -0.3463,  0.4126,  0.0022]])

In [14]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [16]:

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [18]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        text_lengths = text_lengths.cpu()
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 15s
	Train Loss: 0.675 | Train Acc: 56.54%
	 Val. Loss: 0.691 |  Val. Acc: 53.08%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.566 | Train Acc: 69.68%
	 Val. Loss: 0.455 |  Val. Acc: 80.60%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.314 | Train Acc: 87.07%
	 Val. Loss: 0.374 |  Val. Acc: 84.32%
Epoch: 04 | Epoch Time: 0m 15s
	Train Loss: 0.212 | Train Acc: 91.91%
	 Val. Loss: 0.364 |  Val. Acc: 86.32%
Epoch: 05 | Epoch Time: 0m 15s
	Train Loss: 0.156 | Train Acc: 94.43%
	 Val. Loss: 0.367 |  Val. Acc: 86.26%
Epoch: 06 | Epoch Time: 0m 15s
	Train Loss: 0.114 | Train Acc: 96.03%
	 Val. Loss: 0.380 |  Val. Acc: 86.19%
Epoch: 07 | Epoch Time: 0m 15s
	Train Loss: 0.085 | Train Acc: 97.30%
	 Val. Loss: 0.453 |  Val. Acc: 87.00%
Epoch: 08 | Epoch Time: 0m 15s
	Train Loss: 0.066 | Train Acc: 97.88%
	 Val. Loss: 0.447 |  Val. Acc: 87.08%
Epoch: 09 | Epoch Time: 0m 15s
	Train Loss: 0.050 | Train Acc: 98.49%
	 Val. Loss: 0.470 |  Val. Acc: 84.63%
Epoch: 10 | Epoch T

In [22]:

model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.398 | Test Acc: 84.31%


In [23]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [24]:
predict_sentiment(model, "my name is Rohan")

0.4499405026435852

In [25]:

predict_sentiment(model, "Rohan is name my")

0.6206717491149902